### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Kidney_Papillary_Cell_Carcinoma/GSE41282'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# 1. Gene Expression Data Availability
if '!Series_title' in background_info and 'gene expression' in background_info['!Series_title'].lower():
    is_gene_available = True
else:
    is_gene_available = False

# 2. Variable Availability and Data Type Conversion

# Helper function to extract data after ':' and convert from list of strings
def extract_values(values):
    return [val.split(':')[-1].strip() for val in values if pd.notna(val)]

# 2.1 Data Availability
# Extract Key Values
sample_dict = {0: ['tissue source: radical nephrectomy specimen 16A4', 'tissue source: radical nephrectomy specimen 17A5', 'tissue source: radical nephrectomy specimen 22A6', 'tissue source: radical nephrectomy specimen 138', 'tissue source: radical nephrectomy specimen A3', 'tissue source: radical nephrectomy specimen I01', 'tissue source: radical nephrectomy specimen I10', 'tissue source: radical nephrectomy specimen I13', 'tissue source: radical nephrectomy specimen I15', 'tissue source: radical nephrectomy specimen I93', 'tissue source: radical nephrectomy specimen II22', 'tissue source: radical nephrectomy specimen II29', 'tissue source: radical nephrectomy specimen II30', 'tissue source: radical nephrectomy specimen II33', 'tissue source: radical nephrectomy specimen II38', 'tissue source: radical nephrectomy specimen II93', 'tissue source: radical nephrectomy specimen II98', 'tissue source: radical nephrectomy specimen II99', 'tissue source: radical nephrectomy specimen 182', 'tissue source: radical nephrectomy specimen A551', 'tissue source: radical nephrectomy specimen I84', 'tissue source: radical nephrectomy specimen I89', 'tissue source: radical nephrectomy specimen I95'], 1: ['tissue type: normal (non-malignant)', 'tissue type: tumor'], 2: ['sample type: FFPE tissue', 'rcc subtype: clear cell RCC', 'rcc subtype: papillary RCC type 1', 'rcc subtype: papillary RCC type 2'], 3: [nan, 'sample type: FFPE tissue']}
trait_values = extract_values(sample_dict.get(2, []))
age_values = extract_values(sample_dict.get(0, []))  # These are inferred to be sample IDs, which do not provide age
gender_values = extract_values(sample_dict.get(1, []))  # These are tissue types, not related to gender

# Setting the keys based on data availability check
if 'papillary' in ' '.join(trait_values).lower():
    trait_row = 2  # Key for 'Kidney_Papillary_Cell_Carcinoma'
else:
    trait_row = None

# Age and gender are not available with the given sample characteristics
age_row = None
gender_row = None

# 2.3 Data Type Conversion

def convert_trait(value):
    if pd.isna(value):
        return None
    val = value.split(':')[-1].strip().lower()
    if 'clear cell' in val:
        return 0
    elif 'papillary' in val:
        return 1
    return None

def convert_age(value):
    return None  # No age data available

def convert_gender(value):
    return None  # No gender data available

# 3. Save Metadata
save_cohort_info('GSE41282', './preprocessed/Kidney_Papillary_Cell_Carcinoma/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Kidney_Papillary_Cell_Carcinoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/trait_data/GSE41282.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
